# Creando tablas e insertando datos en SQL
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### Introducción

Durante esta actividad vamos a aprender los conceptos básicos de SQL. Vamos a aprender a definir tablas, e insertar, eliminar y actualizar datos de esta tabla.

### Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.

### Outline

En esta actividad aprenderemos a:

- Habilitar un servidor de PostgreSQL en Google colab con un Jupyter notebook.
- Definir esquema de una tabla relacional.
- Insertar datos.
- Crear llaves en las tablas.

### Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`

Que corresponde a películas, actores, y la información de los roles interpretados por una actor en una película.

Las llaves en nuestro caso son:
1. `pid`, para `Peliculas`
2. `aid` para `Actores`
3. `(aid,pid,rol)` para `actuo_en`.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

### DDL vs. DML

Acuerdense que:


*   DDL: Lenguaje de definición de datos
 * Crear y modificar tablas, atributos y llaves
*   DML: Lenguaje de manipulación de datos
 * Consultar una o más tablas
 * Insertar, eliminar, modificar tuplas


Típicamente ocupados juntos.


### Definiendo un esquema

En esta clase queremos crear una base de datos con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`

Para esto, primero necesitamos conocer el tipo de datos que guardará cada atributo de nuestras relaciones.

Al crear las tablas disponemos varios tipos de datos. Por ahora destacamos los siguientes:

- Caracteres (_Strings_):
  - `CHAR(20)`: _Strings_ de largo fijo.
  - `VARCHAR(20)`: _Strings_ de largo variable.
  
- Números:
  - `INT`
  - `FLOAT`
  - `SMALLINT`
  
- Tiempos y fechas:
  - `DATE`: fecha.
  - `TIME`: hora.
  - `TIMESTAMP`: fecha y hora.


Para definir nuestro esquema hay que ejecutar el siguiente comando SQL:

In [ ]:
%%sql

CREATE TABLE Peliculas(
    pid int,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);


**Ojo**: cada vez que anteponemos `%sql` es porque esa línea corresponde a un comando a SQL que va a la base de datos en la que estamos trabajando. Si queremos tener todo un bloque con instrucciones SQL tenemos que usar `%%sql` (habrán ejemplos de esto a lo largo del *notebook*).

En este momento, nuestro servidor cuenta con la tabla Peliculas definida arriba. Si intentamos crear la tabla de nuevo, recibiremos un error.

In [ ]:
%%sql

CREATE TABLE Peliculas(
    pid int,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

Si queremos asegurarnos que no crearemos una tabla de nuevo, siempre podemos validar esto ocupando el siguiente comando:

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS Peliculas(
    pid int,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

Podemos crear las otras dos tablas:


- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`



In [ ]:
%%sql

CREATE TABLE Actores(
    aid int,
    anombre varchar(30),
    aedad int
);

CREATE TABLE Actuo_en(
    aid int,
    pid int,
    rol varchar(30)
);


### Insertar datos en la tabla

Para revisar el contenido de una tabla, podemos correr la consulta `SELECT * FROM NombreTabla` (más de esto en la siguiente clase).

Si ejecutamos la consulta `SELECT * FROM Peliculas` notaremos que el resultado es vacío. Esto ocurre porque todavía no hemos insertado ningun dato a la tabla.

In [ ]:
%sql SELECT * FROM Peliculas;

Para insertar valores, la forma básica es la siguiente:

```SQL
INSERT INTO <Nombre Tabla> 
VALUES (<valor atributo 1> , ..., <valor atributo N>)
```

Por ejemplo si ejecutamos la consulta:

```SQL
INSERT INTO Peliculas 
VALUES(321351, 'V for Vendetta', 2005,'Action', 8.2 ,'James McTeigue')
```

estamos insertando una Pelicula con `pid` 321351, `pnombre` V for Vendetta, etc.

In [ ]:
%sql INSERT INTO Peliculas VALUES(321351, 'V for Vendetta', 2005,'Action', 8.2 ,'James McTeigue')

Al revisar el contenido de la tabla de nuevo, ahora tenemos:

In [ ]:
%sql SELECT * FROM Peliculas;

Siguiendo con este proceso:

In [ ]:
%%sql 

INSERT INTO Peliculas VALUES(321352, 'Batman', 2005,'Action', 8.2 ,'C. Nolan');

SELECT * FROM Peliculas;

Notense que nada prohibe insertar la película Batman con el mismo pid cómo V for Vendetta.

In [ ]:
%%sql 

INSERT INTO Peliculas VALUES(321351, 'Batman', 2005,'Action', 8.2 ,'C. Nolan');

SELECT * FROM Peliculas;

Esto ocurre porque no hemos especificado `pid` como la llave primaria de nuestra tabla. Para especificar las llaves primarias al crear un esqeuma hacemos lo siguiente.

In [ ]:
%%sql

DROP TABLE IF EXISTS Peliculas;
DROP TABLE IF EXISTS Actores;
DROP TABLE IF EXISTS Actuo_En;

CREATE TABLE IF NOT EXISTS Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

CREATE TABLE Actores(
    aid int PRIMARY KEY,
    anombre varchar(30),
    aedad int
);

CREATE TABLE Actuo_en(
    aid int,
    pid int,
    rol varchar(30),
    PRIMARY KEY (aid,pid,rol)
);

Ahora no podemos hacer el mismo error de antes:

In [ ]:
%%sql 

INSERT INTO Peliculas VALUES(321351, 'V for Vendetta', 2005,'Action', 8.2 ,'James McTeigue');
INSERT INTO Peliculas VALUES(321352, 'Batman', 2005,'Action', 8.2 ,'C. Nolan');

SELECT * FROM Peliculas;

In [ ]:
%%sql 

INTO Peliculas VALUES(321351, 'Batman', 2005,'Action', 8.2 ,'C. Nolan');

### Valores default

Muchas veces no sabemos el valor de un atributo al crear la tabla, y por lo tanto lo queremos dejar con una valor por defecto. Por ejemplo, en la tabla `Peliculas`, el valor de la calificación quizás no conocemos al estrenar la película, y lo conseguimos después. Para esto podemos en la definición de nuestro esquema poner el valor default de este atributo en 0. Adicionalmente, podemos especificar, por ejemplo, que si no conocemos la categoría de la película, lo dejamos cómo Drama. Lo podemos hacer en SQL así:

In [ ]:
%%sql

DROP TABLE IF EXISTS Peliculas;

CREATE TABLE Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30) DEFAULT 'Drama',
    pcalificacion float DEFAULT 0,
    pdirector varchar(30)
);

Para insertar los datos con algunos campos faltante, necesitamos nombrar los atributos:

In [ ]:
%%sql

INSERT INTO Peliculas(pid,pnombre,paño,pdirector) VALUES(321351, 'V for Vendetta', 2005,'James McTeigue');

SELECT * FROM Peliculas;

Podemos omitir solo algunos atributos que tienen el valor default:

In [ ]:
%%sql

INSERT INTO Peliculas(pid,pnombre,paño,pcalificacion,pdirector) VALUES(22, 'Batman', 2008,8.5,'C.Nolan');

SELECT * FROM Peliculas;

### Resumen

En este tutorial aprendimos cómo definir un esquema relacional en el motor SQL Postgres, y cómo llenar las tablas con datos. En próximas clases aprenderemos cómo consultar a nuestra base de datos. 

#Materia adicional

### Modificando tablas

En SQL es posible modificar tablas. Por ejemplo si quisieramos eliminar el atributo `pdirector` de la tabla `Peliculas`, podríamos ejecutar la consulta:

In [ ]:
%%sql

ALTER TABLE Peliculas DROP COLUMN pdirector;

SELECT * FROM Peliculas;



Similarmente, si al definir el esquema nos olvidamos de un atributo, lo podemos agregar (incluso con un valor por defecto):

In [ ]:
%%sql

ALTER TABLE Peliculas ADD COLUMN productor varchar(30) DEFAULT 'Warner Bros';

SELECT * FROM Peliculas;

In [ ]:
%%sql

ALTER TABLE Peliculas ADD COLUMN director varchar(30);

SELECT * FROM Peliculas;

Finalmente, para borrar una tabla, ocupamos:

In [ ]:
%%sql

DROP TABLE Peliculas;

Al consultar esta tabla ahora, recibimos un error:

In [ ]:
%%sql

SELECT * FROM Peliculas;